In [1]:
from pathlib import Path
import pandas as pd
import sklearn.metrics
import numpy as np
from tqdm import tqdm
import geopandas as gpd
import seaborn as sns

# Out Directory

In [2]:
across_sites_dir = Path('out/_all_sites_aggregated')
across_sites_dir.mkdir(exist_ok=True, parents=True)

# Sites

In [3]:
df_site_burst = gpd.read_file('data/val_sites_subset.geojson')
df_site_burst.head()

,site_id,change_label,change_type,change_time,last_observation_time,mgrs_tile_id,jpl_burst_id,geometry
0,4,VLmaj,Fire,2021-11-07,2021-11-02,34NCL,T036-076228-IW3,POINT (19.33289 4.57490)
1,13,VLmaj,Crop cycle change,2021-10-03,NaT,39STR,T108-230715-IW2,POINT (48.77698 31.75490)
2,13,VLmaj,Crop cycle change,2021-10-03,NaT,39STR,T101-214979-IW1,POINT (48.77698 31.75490)
3,22,VLmin,Natural browning,2021-12-19,2021-12-12,37SBV,T014-028130-IW2,POINT (36.61178 35.52891)
4,22,VLmin,Natural browning,2021-12-19,2021-12-12,37SBV,T021-043823-IW2,POINT (36.61178 35.52891)


In [4]:
df_site = df_site_burst.drop_duplicates(subset=['site_id'], keep='first')[['site_id', 'change_label', 'change_type']]
df_site.head()

,site_id,change_label,change_type
0,4,VLmaj,Fire
1,13,VLmaj,Crop cycle change
3,22,VLmin,Natural browning
6,30,noChange,No Change
7,34,VLmin,Wetland


# Get All Tables

In [5]:
metric_paths = sorted(list(Path('out').glob('*/*/metrics_ts/*.parquet')))
metric_paths[:3]

[PosixPath('out/site_104/T144-308010-IW2/metrics_ts/cusum_prob_max.parquet'),
 PosixPath('out/site_104/T144-308010-IW2/metrics_ts/cusum_vh.parquet'),
 PosixPath('out/site_104/T144-308010-IW2/metrics_ts/log_ratio_vh.parquet')]

In [6]:
metric_names = [p.stem for p in metric_paths]
metric_names[:3]

['cusum_prob_max', 'cusum_vh', 'log_ratio_vh']

In [7]:
burst_ids = [p.parents[1].stem for p in metric_paths]
burst_ids[:2]

['T144-308010-IW2', 'T144-308010-IW2']

In [8]:
site_ids_rtc = [int(p.parents[2].stem.split('_')[1]) for p in metric_paths]
site_ids_rtc[:2]

[104, 104]

In [9]:
dfs_rtc = [pd.read_parquet(p) for p in metric_paths]
dfs_rtc[0].head()

,site_id,burst_id,acq_date_post,metric,metric_90m,val_obs,rtc_changes_aggressive,rtc_changes_aggressive_confirmed,rtc_changes_conservative,rtc_changes_conservative_confirmed,val_change_aggressive,val_change_aggressive_confirmed,val_change_conservative,val_change_conservative_confirmed
0,104,T144-308010-IW2,2021-10-20 13:59:16,0.614765,0.504232,noChange,0,0,0,0,0,0,0,0
1,104,T144-308010-IW2,2021-10-26 14:00:16,1.000000,1.000000,noChange,1,1,1,1,0,0,0,0
2,104,T144-308010-IW2,2021-11-01 13:59:16,0.967204,0.954869,noChange,1,1,1,1,0,0,0,0
3,104,T144-308010-IW2,2021-11-07 14:00:16,0.976512,0.861402,noChange,1,1,1,1,0,0,0,0
4,104,T144-308010-IW2,2021-11-13 13:59:16,0.729506,0.645800,noChange,0,0,0,0,0,0,0,0


In [10]:
hls_paths = [Path(f'data/dist_hls_timeseries_by_site/site_{site_id}.parquet') for site_id in set(site_ids_rtc)]
assert all([p.exists() for p in hls_paths])
assert len(hls_paths) == 59

In [12]:
dfs_hls = [pd.read_parquet(p) for p in hls_paths]
dfs_hls[0].head()

,site_id,granule_id,sensing_time,veg_status,gen_status,date,val_obs,val_change_conservative,val_change_aggressive,dist-hls-veg_change_conservative,dist-hls-veg_change_aggressive,dist-hls-gen_change_conservative,dist-hls-gen_change_aggressive,val_change_conservative_confirmed,val_change_aggressive_confirmed,dist-hls-veg_change_conservative_confirmed,dist-hls-veg_change_aggressive_confirmed,dist-hls-gen_change_conservative_confirmed,dist-hls-gen_change_aggressive_confirmed
0,256,OPERA_L3_DIST-ALERT-HLS_T30UUA_20211001T112121...,2021-10-01,0,0,2021-10-01,noObs,255,255,0,0,0,0,255,255,0,0,0,0
1,256,OPERA_L3_DIST-ALERT-HLS_T30UUA_20211006T112119...,2021-10-06,255,255,2021-10-06,noChange,0,0,0,0,0,0,0,0,0,0,0,0
2,256,OPERA_L3_DIST-ALERT-HLS_T30UUA_20211009T110533...,2021-10-09,255,255,2021-10-09,noChange,0,0,0,0,0,0,0,0,0,0,0,0
3,256,OPERA_L3_DIST-ALERT-HLS_T30UUA_20211011T112121...,2021-10-11,255,255,2021-10-11,noChange,0,0,0,0,0,0,0,0,0,0,0,0
4,256,OPERA_L3_DIST-ALERT-HLS_T30UUA_20211014T113321...,2021-10-14,0,0,2021-10-14,noChange,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
lll = [df['dist-hls-veg_change_conservative_confirmed'].sum() for df in dfs_hls]
np.mean(lll)

0.05084745762711865

In [14]:
def update_obs_labels(df: pd.DataFrame) -> pd.DataFrame:
    df['val_obs'] = df['val_obs'].map(lambda obs: obs if obs != 'OCtot' else 'OCmaj')
    df['val_obs'] = df['val_obs'].map(lambda obs: obs if obs not in [0, '0'] else 'noObs')
    return df

dfs_rtc = list(map(update_obs_labels, dfs_rtc))
dfs_hls = list(map(update_obs_labels, dfs_hls))

In [15]:
class_dict = {0: 'undisturbed',
              1: 'disturbed'}

In [16]:
def get_commission_error(y_val, y_pred, label) -> float:
    fp = np.sum((y_val != label) & (y_pred == label)).astype('float32')
    tp = np.sum((y_val == label) & (y_pred == label)).astype('float32')

    denom = tp + fp
    if denom == 0:
        return 0.
    ce = fp / denom
    return ce


def get_ommission_error(y_val, y_pred, label) -> float:
    fn = np.sum((y_val == label) & (y_pred != label)).astype('float32')
    tp = np.sum((y_val == label) & (y_pred == label)).astype('float32')

    denom = tp + fn
    if denom == 0:
        return 0.
    oe = fn / denom
    return oe

def get_total_accuracy(y_val, y_pred) -> float:
    if y_val.size == 0:
        return 1.
    acc = (y_val == y_pred).sum() / (y_val.size)
    return acc

def get_accuracy_metrics_for_one_pair(y_val, y_rtc):
    mask = (y_rtc == 255) | (y_val == 255)
    y_rtc_ = y_rtc[~mask]
    y_val_ = y_val[~mask]
    prec, recall, f1, supp = sklearn.metrics.precision_recall_fscore_support(y_val_,
                                                                             y_rtc_,
                                                                             labels=[0, 1],
                                                                             # if there are no classes
                                                                             # Assume "perfect"
                                                                             zero_division=1
                                                                             )
    recall_per_class = {class_dict[label]: recall[label] for label in [0, 1]}
    prec_per_class = {class_dict[label]: prec[label] for label in [0, 1]}
    f1_per_class = {class_dict[label]: f1[label] for label in [0, 1]}
    supp_per_class = {class_dict[label]: int(supp[label]) for label in [0, 1]}

    n = len(y_rtc_)
    d = y_rtc_.sum()
    supp_prod_change_per_class = {'disturbed': d, 'undisturbed': n - d}
    supp_prod_change_per_class_perc = {'disturbed': d / n,
                                       'undisturbed': 1 - d/n}
    
    co_err = {'disturbed': get_commission_error(y_val_, y_rtc_, 1),
                'undisturbed': get_commission_error(y_val_, y_rtc_, 0)}
    om_err = {'disturbed': get_ommission_error(y_val_, y_rtc_, 1),
               'undisturbed': get_ommission_error(y_val_, y_rtc_, 0)}
    jacc = sklearn.metrics.jaccard_score(y_val_, y_rtc_, pos_label=1, zero_division=1)
    return {
            'precision': prec_per_class,
            'recall': recall_per_class,
            'f1_score': f1_per_class,
            'supp_val': supp_per_class,
            'supp_prod_change': supp_prod_change_per_class,
            'supp_prod_change_perc': supp_prod_change_per_class_perc,
            'commission_error': co_err,
            'ommission_error': om_err,
            'jaccard_score': jacc,
            'total_samples': sum(supp_per_class.values()),
            'total_accuracy': get_total_accuracy(y_val_, y_rtc_)}
    

In [17]:
get_accuracy_metrics_for_one_pair(dfs_hls[0].val_change_aggressive_confirmed, dfs_hls[0]['dist-hls-veg_change_conservative'])

{'precision': {'undisturbed': 0.40131578947368424, 'disturbed': 1.0},
 'recall': {'undisturbed': 1.0, 'disturbed': 0.061855670103092786},
 'f1_score': {'undisturbed': 0.5727699530516432,
  'disturbed': 0.11650485436893204},
 'supp_val': {'undisturbed': 61, 'disturbed': 97},
 'supp_prod_change': {'disturbed': 6, 'undisturbed': 152},
 'supp_prod_change_perc': {'disturbed': 0.0379746835443038,
  'undisturbed': 0.9620253164556962},
 'commission_error': {'disturbed': 0.0, 'undisturbed': 0.5986842},
 'ommission_error': {'disturbed': 0.9381443, 'undisturbed': 0.0},
 'jaccard_score': 0.061855670103092786,
 'total_samples': 158,
 'total_accuracy': 0.4240506329113924}

In [18]:
get_accuracy_metrics_for_one_pair(dfs_rtc[0].val_change_aggressive_confirmed, dfs_rtc[0].rtc_changes_conservative_confirmed)

{'precision': {'undisturbed': 1.0, 'disturbed': 0.0},
 'recall': {'undisturbed': 0.9090909090909091, 'disturbed': 1.0},
 'f1_score': {'undisturbed': 0.9523809523809523, 'disturbed': 0.0},
 'supp_val': {'undisturbed': 33, 'disturbed': 0},
 'supp_prod_change': {'disturbed': 3, 'undisturbed': 30},
 'supp_prod_change_perc': {'disturbed': 0.09090909090909091,
  'undisturbed': 0.9090909090909091},
 'commission_error': {'disturbed': 1.0, 'undisturbed': 0.0},
 'ommission_error': {'disturbed': 0.0, 'undisturbed': 0.09090909},
 'jaccard_score': 0.0,
 'total_samples': 33,
 'total_accuracy': 0.9090909090909091}

In [19]:
def get_accuracy_metrics_for_all_pairs(df: pd.DataFrame, sensor='sar'):
    if sensor not in ['optical', 'sar']:
        raise ValueError
    out = {}
    sensor_column_tokens = ['rtc_changes'] if sensor == 'sar' else ['dist-hls-veg_change', 'dist-hls-gen_change']
    sensor_serial_tokens = ['rtc'] if sensor == 'sar' else ['dist-hls-veg', 'dist-hls-gen']
    site_id = df.site_id[0]
    for sensor_column_token, serial_sensor_token in zip(sensor_column_tokens, sensor_serial_tokens):
        for val_token in ['conservative', 'aggressive']:
            for change_strategy_token in ['conservative', 'aggressive']:
                y_val = df[f'val_change_{val_token}_confirmed']
                y_rtc = df[f'{sensor_column_token}_{change_strategy_token}_confirmed']
                acc_dict = get_accuracy_metrics_for_one_pair(y_val, y_rtc)
                acc_dict['site_id'] = site_id
                acc_dict['change_category'] = serial_sensor_token
                
                out.update({f'val-{val_token}-confirmed__{serial_sensor_token}-{change_strategy_token}-confirmed': acc_dict})
    return out

In [20]:
get_accuracy_metrics_for_all_pairs(dfs_rtc[0], sensor='sar')

{'val-conservative-confirmed__rtc-conservative-confirmed': {'precision': {'undisturbed': 1.0,
   'disturbed': 0.0},
  'recall': {'undisturbed': 0.9090909090909091, 'disturbed': 1.0},
  'f1_score': {'undisturbed': 0.9523809523809523, 'disturbed': 0.0},
  'supp_val': {'undisturbed': 33, 'disturbed': 0},
  'supp_prod_change': {'disturbed': 3, 'undisturbed': 30},
  'supp_prod_change_perc': {'disturbed': 0.09090909090909091,
   'undisturbed': 0.9090909090909091},
  'commission_error': {'disturbed': 1.0, 'undisturbed': 0.0},
  'ommission_error': {'disturbed': 0.0, 'undisturbed': 0.09090909},
  'jaccard_score': 0.0,
  'total_samples': 33,
  'total_accuracy': 0.9090909090909091,
  'site_id': 104,
  'change_category': 'rtc'},
 'val-conservative-confirmed__rtc-aggressive-confirmed': {'precision': {'undisturbed': 1.0,
   'disturbed': 0.0},
  'recall': {'undisturbed': 0.9090909090909091, 'disturbed': 1.0},
  'f1_score': {'undisturbed': 0.9523809523809523, 'disturbed': 0.0},
  'supp_val': {'undistu

In [21]:
get_accuracy_metrics_for_all_pairs(dfs_hls[0], sensor='optical')

{'val-conservative-confirmed__dist-hls-veg-conservative-confirmed': {'precision': {'undisturbed': 0.42038216560509556,
   'disturbed': 1.0},
  'recall': {'undisturbed': 1.0, 'disturbed': 0.0},
  'f1_score': {'undisturbed': 0.5919282511210763, 'disturbed': 0.0},
  'supp_val': {'undisturbed': 66, 'disturbed': 91},
  'supp_prod_change': {'disturbed': 0, 'undisturbed': 157},
  'supp_prod_change_perc': {'disturbed': 0.0, 'undisturbed': 1.0},
  'commission_error': {'disturbed': 0.0, 'undisturbed': 0.57961786},
  'ommission_error': {'disturbed': 1.0, 'undisturbed': 0.0},
  'jaccard_score': 0.0,
  'total_samples': 157,
  'total_accuracy': 0.42038216560509556,
  'site_id': 256,
  'change_category': 'dist-hls-veg'},
 'val-conservative-confirmed__dist-hls-veg-aggressive-confirmed': {'precision': {'undisturbed': 0.44594594594594594,
   'disturbed': 1.0},
  'recall': {'undisturbed': 1.0, 'disturbed': 0.0989010989010989},
  'f1_score': {'undisturbed': 0.616822429906542, 'disturbed': 0.18},
  'supp_v

In [22]:
acc_data_rtc = list(map(get_accuracy_metrics_for_all_pairs, tqdm(dfs_rtc[:])))
acc_data_hls = [get_accuracy_metrics_for_all_pairs(df, sensor='optical') for df in tqdm(dfs_hls)]

100%|█████████████████████████████████████████| 59/59 [00:00<00:00, 66.93it/s]


# Aggregate Over All Sites

And a single pair of strategies

## all disturbance types

In [36]:
STRAT_KEY = 'val-conservative-confirmed__rtc-conservative-confirmed'
DISTURBED_METRIC_KEYS = ['precision', 'recall', 'f1_score', 'commission_error', 'ommission_error',
                         'supp_val', 'supp_prod_change', 
                         'supp_prod_change_perc']
OTHER_KEYS = ['jaccard_score', 'total_samples', 'total_accuracy']

assert(all([len(acc_data_rtc) == len(l) for l in [burst_ids, site_ids_rtc, metric_names]]))
disturbed_acc_data_rtc = [{**{'metric': metric},
                           **{'site_id': site_id},
                           **{'burst_id': burst_id},
                           **{key: item[STRAT_KEY][key] for key in OTHER_KEYS}, 
                           **{key: item[STRAT_KEY][key]['disturbed'] for key in DISTURBED_METRIC_KEYS}}
                          for item, metric, burst_id, site_id in zip(acc_data_rtc, metric_names, burst_ids, site_ids_rtc)]

In [37]:
HLS_STRAT_KEY = 'val-conservative-confirmed__dist-hls-gen-conservative-confirmed'

disturbed_acc_data_hls_gen = [{**{'metric': item[HLS_STRAT_KEY]['change_category']},
                               **{'site_id': item[HLS_STRAT_KEY]['site_id']},
                               **{key: item[HLS_STRAT_KEY][key] for key in OTHER_KEYS}, 
                               **{key: item[HLS_STRAT_KEY][key]['disturbed'] for key in DISTURBED_METRIC_KEYS}}
                              for item in acc_data_hls]

In [38]:
HLS_STRAT_KEY = 'val-conservative-confirmed__dist-hls-veg-conservative-confirmed'

disturbed_acc_data_hls_veg = [{**{'metric': item[HLS_STRAT_KEY]['change_category']},
                               **{'site_id': item[HLS_STRAT_KEY]['site_id']},
                               **{key: item[HLS_STRAT_KEY][key] for key in OTHER_KEYS}, 
                               **{key: item[HLS_STRAT_KEY][key]['disturbed'] for key in DISTURBED_METRIC_KEYS}}
                              for item in acc_data_hls]

In [42]:
df_dist_rtc = pd.DataFrame(disturbed_acc_data_rtc)
df_dist_rtc.head()

,metric,site_id,burst_id,jaccard_score,total_samples,total_accuracy,precision,recall,f1_score,commission_error,ommission_error,supp_val,supp_prod_change,supp_prod_change_perc
0,cusum_prob_max,104,T144-308010-IW2,0.0,33,0.909091,0.0,1.0,0.0,1.0,0.0,0,3,0.090909
1,cusum_vh,104,T144-308010-IW2,1.0,33,1.000000,1.0,1.0,1.0,0.0,0.0,0,0,0.000000
2,log_ratio_vh,104,T144-308010-IW2,1.0,33,1.000000,1.0,1.0,1.0,0.0,0.0,0,0,0.000000
3,mahalanobis_1d_max,104,T144-308010-IW2,0.0,33,0.000000,0.0,1.0,0.0,1.0,0.0,0,33,1.000000
4,mahalanobis_2d,104,T144-308010-IW2,1.0,33,1.000000,1.0,1.0,1.0,0.0,0.0,0,0,0.000000


In [43]:
df_dist_hls_gen = pd.DataFrame(disturbed_acc_data_hls_gen)
df_dist_hls_gen.head()

,metric,site_id,jaccard_score,total_samples,total_accuracy,precision,recall,f1_score,commission_error,ommission_error,supp_val,supp_prod_change,supp_prod_change_perc
0,dist-hls-gen,256,0.0,157,0.420382,1.0,0.0,0.0,0.0,1.0,91,0,0.0
1,dist-hls-gen,4,1.0,179,1.000000,1.0,1.0,1.0,0.0,0.0,0,0,0.0
2,dist-hls-gen,261,0.0,168,0.547619,1.0,0.0,0.0,0.0,1.0,76,0,0.0
3,dist-hls-gen,263,1.0,208,1.000000,1.0,1.0,1.0,0.0,0.0,0,0,0.0
4,dist-hls-gen,264,1.0,209,1.000000,1.0,1.0,1.0,0.0,0.0,0,0,0.0


In [44]:
df_dist_hls_veg = pd.DataFrame(disturbed_acc_data_hls_veg)
df_dist_hls_veg.head()

,metric,site_id,jaccard_score,total_samples,total_accuracy,precision,recall,f1_score,commission_error,ommission_error,supp_val,supp_prod_change,supp_prod_change_perc
0,dist-hls-veg,256,0.0,157,0.420382,1.0,0.0,0.0,0.0,1.0,91,0,0.0
1,dist-hls-veg,4,1.0,179,1.000000,1.0,1.0,1.0,0.0,0.0,0,0,0.0
2,dist-hls-veg,261,0.0,168,0.547619,1.0,0.0,0.0,0.0,1.0,76,0,0.0
3,dist-hls-veg,263,1.0,208,1.000000,1.0,1.0,1.0,0.0,0.0,0,0,0.0
4,dist-hls-veg,264,1.0,209,1.000000,1.0,1.0,1.0,0.0,0.0,0,0,0.0


In [45]:
AGG_COLUMNS = OTHER_KEYS + DISTURBED_METRIC_KEYS

In [46]:
df_dist = pd.concat([df_dist_rtc, df_dist_hls_gen, df_dist_hls_veg])
df_dist.head()

,metric,site_id,burst_id,jaccard_score,total_samples,total_accuracy,precision,recall,f1_score,commission_error,ommission_error,supp_val,supp_prod_change,supp_prod_change_perc
0,cusum_prob_max,104,T144-308010-IW2,0.0,33,0.909091,0.0,1.0,0.0,1.0,0.0,0,3,0.090909
1,cusum_vh,104,T144-308010-IW2,1.0,33,1.000000,1.0,1.0,1.0,0.0,0.0,0,0,0.000000
2,log_ratio_vh,104,T144-308010-IW2,1.0,33,1.000000,1.0,1.0,1.0,0.0,0.0,0,0,0.000000
3,mahalanobis_1d_max,104,T144-308010-IW2,0.0,33,0.000000,0.0,1.0,0.0,1.0,0.0,0,33,1.000000
4,mahalanobis_2d,104,T144-308010-IW2,1.0,33,1.000000,1.0,1.0,1.0,0.0,0.0,0,0,0.000000


In [47]:
df_dist_agg_all = df_dist[['metric'] + AGG_COLUMNS].groupby('metric').agg(['mean', 'std'])
df_dist_agg_all

jaccard_score           total_samples             \
                            mean       std          mean        std   
metric                                                                
cusum_prob_max          0.726558  0.438487     28.773585   7.030929   
cusum_vh                0.773585  0.420499     28.754717   7.028935   
dist-hls-gen            0.711864  0.456782    184.661017  38.038823   
dist-hls-veg            0.728814  0.448388    184.661017  38.038823   
log_ratio_vh            0.593173  0.474284     28.764151   7.032648   
mahalanobis_1d_max      0.079732  0.194978     28.754717   7.028935   
mahalanobis_2d          0.696281  0.441653     28.773585   7.030929   
mahalanobis_vh          0.079732  0.194978     28.754717   7.028935   
transformer             0.787703  0.396430     28.660377   7.139277   

                   total_accuracy           precision              recall  \
                             mean       std      mean       std      mean   
metric                                                                      
cusum_prob_max           0.902705  0.212813  0.876101  0.321068  0.810091   
cusum_vh                 0.920268  0.194978  1.000000  0.000000  0.773585   
dist-hls-gen             0.911160  0.207094  0.966102  0.182521  0.728814   
dist-hls-veg             0.911911  0.206077  1.000000  0.000000  0.728814   
log_ratio_vh             0.874014  0.202545  0.677401  0.459773  0.822947   
mahalanobis_1d_max       0.079732  0.194978  0.079732  0.194978  1.000000   
mahalanobis_2d           0.895327  0.205529  0.784119  0.404937  0.836111   
mahalanobis_vh           0.079732  0.194978  0.079732  0.194978  1.000000   
transformer              0.925396  0.176141  0.955882  0.196997  0.811688   

                              ... commission_error           ommission_error  \
                         std  ...             mean       std            mean   
metric                        ...                                              
cusum_prob_max      0.389103  ...         0.123899  0.321068        0.189909   
cusum_vh            0.420499  ...         0.000000  0.000000        0.226415   
dist-hls-gen        0.448388  ...         0.033898  0.182521        0.271186   
dist-hls-veg        0.448388  ...         0.000000  0.000000        0.271186   
log_ratio_vh        0.363139  ...         0.322599  0.459773        0.177053   
mahalanobis_1d_max  0.000000  ...         0.920268  0.194978        0.000000   
mahalanobis_2d      0.354937  ...         0.215881  0.404937        0.163889   
mahalanobis_vh      0.000000  ...         0.920268  0.194978        0.000000   
transformer         0.380763  ...         0.044118  0.196997        0.188312   

                               supp_val            supp_prod_change            \
                         std       mean        std             mean       std   
metric                                                                          
cusum_prob_max      0.389103   2.169811   5.483259         0.716981  1.916121   
cusum_vh            0.420499   2.169811   5.483259         0.000000  0.000000   
dist-hls-gen        0.448388  15.864407  38.694853         0.152542  0.867290   
dist-hls-veg        0.448388  15.864407  38.694853         0.000000  0.000000   
log_ratio_vh        0.363139   2.169811   5.483259         2.311321  4.216438   
mahalanobis_1d_max  0.000000   2.169811   5.483259        28.754717  7.028935   
mahalanobis_2d      0.354937   2.169811   5.483259         1.471698  3.420408   
mahalanobis_vh      0.000000   2.169811   5.483259        28.754717  7.028935   
transformer         0.380763   2.169811   5.483259         0.490566  2.029522   

                   supp_prod_change_perc            
                                    mean       std  
metric                                              
cusum_prob_max                  0.032859  0.114995  
cusum_vh                        0.000000  0.000000  
dist-hls-gen                    0.000751  

In [64]:
unique_metrics = sorted(list(set(metric_names)))
METRIC_ORDER = ['dist-hls-veg', 'dist-hls-gen'] + unique_metrics

In [65]:
df_dist_agg_all_count = df_dist[['metric'] + AGG_COLUMNS].groupby(['metric']).size().rename(index="count")#.reset_index(drop=False)
df_dist_agg_all['count'] = df_dist_agg_all_count
df_dist_agg_all = df_dist_agg_all.reindex(METRIC_ORDER)
df_dist_agg_all

jaccard_score           total_samples             \
                            mean       std          mean        std   
metric                                                                
dist-hls-veg            0.728814  0.448388    184.661017  38.038823   
dist-hls-gen            0.711864  0.456782    184.661017  38.038823   
cusum_prob_max          0.726558  0.438487     28.773585   7.030929   
cusum_vh                0.773585  0.420499     28.754717   7.028935   
log_ratio_vh            0.593173  0.474284     28.764151   7.032648   
mahalanobis_1d_max      0.079732  0.194978     28.754717   7.028935   
mahalanobis_2d          0.696281  0.441653     28.773585   7.030929   
mahalanobis_vh          0.079732  0.194978     28.754717   7.028935   
transformer             0.787703  0.396430     28.660377   7.139277   

                   total_accuracy           precision              recall  \
                             mean       std      mean       std      mean   
metric                                                                      
dist-hls-veg             0.911911  0.206077  1.000000  0.000000  0.728814   
dist-hls-gen             0.911160  0.207094  0.966102  0.182521  0.728814   
cusum_prob_max           0.902705  0.212813  0.876101  0.321068  0.810091   
cusum_vh                 0.920268  0.194978  1.000000  0.000000  0.773585   
log_ratio_vh             0.874014  0.202545  0.677401  0.459773  0.822947   
mahalanobis_1d_max       0.079732  0.194978  0.079732  0.194978  1.000000   
mahalanobis_2d           0.895327  0.205529  0.784119  0.404937  0.836111   
mahalanobis_vh           0.079732  0.194978  0.079732  0.194978  1.000000   
transformer              0.925396  0.176141  0.955882  0.196997  0.811688   

                              ... commission_error ommission_error            \
                         std  ...              std            mean       std   
metric                        ...                                              
dist-hls-veg        0.448388  ...         0.000000        0.271186  0.448388   
dist-hls-gen        0.448388  ...         0.182521        0.271186  0.448388   
cusum_prob_max      0.389103  ...         0.321068        0.189909  0.389103   
cusum_vh            0.420499  ...         0.000000        0.226415  0.420499   
log_ratio_vh        0.363139  ...         0.459773        0.177053  0.363139   
mahalanobis_1d_max  0.000000  ...         0.194978        0.000000  0.000000   
mahalanobis_2d      0.354937  ...         0.404937        0.163889  0.354937   
mahalanobis_vh      0.000000  ...         0.194978        0.000000  0.000000   
transformer         0.380763  ...         0.196997        0.188312  0.380763   

                     supp_val            supp_prod_change            \
                         mean        std             mean       std   
metric                                                                
dist-hls-veg        15.864407  38.694853         0.000000  0.000000   
dist-hls-gen        15.864407  38.694853         0.152542  0.867290   
cusum_prob_max       2.169811   5.483259         0.716981  1.916121   
cusum_vh             2.169811   5.483259         0.000000  0.000000   
log_ratio_vh         2.169811   5.483259         2.311321  4.216438   
mahalanobis_1d_max   2.169811   5.483259        28.754717  7.028935   
mahalanobis_2d       2.169811   5.483259         1.471698  3.420408   
mahalanobis_vh       2.169811   5.483259        28.754717  7.028935   
transformer          2.169811   5.483259         0.490566  2.029522   

                   supp_prod_change_perc           count  
                                    mean       std        
metric                                                    
dist-hls-veg                    0.000000  0.000000    59  
dist-hls-gen                    0.000751  0.004264    59  
cusum_prob_max                  0.032859  0.114995   106  
cusum_vh                        0.000000  0.000000   106  
log_ratio_vh  

In [66]:
cm = sns.light_palette("blue", as_cmap=True)
df_dist_agg_all_styled = df_dist_agg_all.style.background_gradient(cmap=cm)
df_dist_agg_all_styled

In [67]:
df_dist_agg_all_styled.to_excel(across_sites_dir / 'dist_all_sites.xlsx')

## By Disturbance type

In [52]:
df_dist_m = pd.merge(df_site, df_dist, on='site_id', how = 'right')
df_dist_m.head()

,site_id,change_label,change_type,metric,burst_id,jaccard_score,total_samples,total_accuracy,precision,recall,f1_score,commission_error,ommission_error,supp_val,supp_prod_change,supp_prod_change_perc
0,104,noChange,No Change,cusum_prob_max,T144-308010-IW2,0.0,33,0.909091,0.0,1.0,0.0,1.0,0.0,0,3,0.090909
1,104,noChange,No Change,cusum_vh,T144-308010-IW2,1.0,33,1.000000,1.0,1.0,1.0,0.0,0.0,0,0,0.000000
2,104,noChange,No Change,log_ratio_vh,T144-308010-IW2,1.0,33,1.000000,1.0,1.0,1.0,0.0,0.0,0,0,0.000000
3,104,noChange,No Change,mahalanobis_1d_max,T144-308010-IW2,0.0,33,0.000000,0.0,1.0,0.0,1.0,0.0,0,33,1.000000
4,104,noChange,No Change,mahalanobis_2d,T144-308010-IW2,1.0,33,1.000000,1.0,1.0,1.0,0.0,0.0,0,0,0.000000


In [53]:
df_dist_agg_by_type = df_dist_m[['change_type', 'metric'] + AGG_COLUMNS].groupby(['change_type', 'metric']).agg(['mean', 'std'])
df_dist_agg_by_type

jaccard_score     total_samples      \
                                        mean std          mean std   
change_type metric                                                   
Clear cut   cusum_prob_max          0.636364 NaN          23.0 NaN   
            cusum_vh                0.000000 NaN          23.0 NaN   
            dist-hls-gen            0.000000 NaN         204.0 NaN   
            dist-hls-veg            0.000000 NaN         204.0 NaN   
            log_ratio_vh            0.636364 NaN          23.0 NaN   
...                                      ...  ..           ...  ..   
Wetland     log_ratio_vh            0.000000 NaN          32.0 NaN   
            mahalanobis_1d_max      0.000000 NaN          32.0 NaN   
            mahalanobis_2d          1.000000 NaN          32.0 NaN   
            mahalanobis_vh          0.000000 NaN          32.0 NaN   
            transformer             1.000000 NaN          32.0 NaN   

                               total_accuracy     precision        recall      \
                                         mean std      mean std      mean std   
change_type metric                                                              
Clear cut   cusum_prob_max           0.826087 NaN       1.0 NaN  0.636364 NaN   
            cusum_vh                 0.521739 NaN       1.0 NaN  0.000000 NaN   
            dist-hls-gen             0.441176 NaN       0.0 NaN  0.000000 NaN   
            dist-hls-veg             0.470588 NaN       1.0 NaN  0.000000 NaN   
            log_ratio_vh             0.826087 NaN       1.0 NaN  0.636364 NaN   
...                                       ...  ..       ...  ..       ...  ..   
Wetland     log_ratio_vh             0.843750 NaN       0.0 NaN  1.000000 NaN   
            mahalanobis_1d_max       0.000000 NaN       0.0 NaN  1.000000 NaN   
            mahalanobis_2d           1.000000 NaN       1.0 NaN  1.000000 NaN   
            mahalanobis_vh           0.000000 NaN       0.0 NaN  1.000000 NaN   
            transformer              1.000000 NaN       1.0 NaN  1.000000 NaN   

                                ... commission_error     ommission_error      \
                                ...             mean std            mean std   
change_type metric              ...                                            
Clear cut   cusum_prob_max      ...              0.0 NaN        0.363636 NaN   
            cusum_vh            ...              0.0 NaN        1.000000 NaN   
            dist-hls-gen        ...              1.0 NaN        1.000000 NaN   
            dist-hls-veg        ...              0.0 NaN        1.000000 NaN   
            log_ratio_vh        ...              0.0 NaN        0.363636 NaN   
...                             ...              ...  ..             ...  ..   
Wetland     log_ratio_vh        ...              1.0 NaN        0.000000 NaN   
            mahalanobis_1d_max  ...              1.0 NaN        0.000000 NaN   
            mahalanobis_2d      ...              0.0 NaN        0.000000 NaN   
            mahalanobis_vh      ...              1.0 NaN        0.000000 NaN   
            transformer         ...              0.0 NaN        0.000000 NaN   

                               supp_val     supp_prod_change      \
                                   mean std             mean std   
change_type metric                                                 
Clear cut   cusum_prob_max         11.0 NaN              7.0 NaN   
            cusum_vh               11.0 NaN              0.0 NaN   
            dist-hls-gen          108.0 NaN              6.0 NaN   
            dist-hls-veg          108.0 NaN              0.0 NaN   
            log_ratio_vh           11.0 NaN              7.0 NaN   
...                                 ...  ..              ...  ..   
Wetland     log_ratio_vh            0.0 NaN              5.0 NaN   
            mahalanobis_1d_max      0.0 NaN             32.0 NaN   
            mahalanobis_2d          0.0 NaN   

In [54]:
df_dist_agg_by_type_count = df_dist_m[['change_type', 'metric'] + AGG_COLUMNS].groupby(['change_type', 'metric']).size().rename(index="count")#.reset_index(drop=False)
df_dist_agg_by_type_count.head()

change_type  metric        
Clear cut    cusum_prob_max    1
             cusum_vh          1
             dist-hls-gen      1
             dist-hls-veg      1
             log_ratio_vh      1
Name: count, dtype: int64

In [55]:
df_dist_agg_by_type['count'] = df_dist_agg_by_type_count
df_dist_agg_by_type.head()

jaccard_score     total_samples     total_accuracy  \
                                    mean std          mean std           mean   
change_type metric                                                              
Clear cut   cusum_prob_max      0.636364 NaN          23.0 NaN       0.826087   
            cusum_vh            0.000000 NaN          23.0 NaN       0.521739   
            dist-hls-gen        0.000000 NaN         204.0 NaN       0.441176   
            dist-hls-veg        0.000000 NaN         204.0 NaN       0.470588   
            log_ratio_vh        0.636364 NaN          23.0 NaN       0.826087   

                               precision        recall      ...  \
                           std      mean std      mean std  ...   
change_type metric                                          ...   
Clear cut   cusum_prob_max NaN       1.0 NaN  0.636364 NaN  ...   
            cusum_vh       NaN       1.0 NaN  0.000000 NaN  ...   
            dist-hls-gen   NaN       0.0 NaN  0.000000 NaN  ...   
            dist-hls-veg   NaN       1.0 NaN  0.000000 NaN  ...   
            log_ratio_vh   NaN       1.0 NaN  0.636364 NaN  ...   

                           commission_error ommission_error     supp_val      \
                                        std            mean std     mean std   
change_type metric                                                             
Clear cut   cusum_prob_max              NaN        0.363636 NaN     11.0 NaN   
            cusum_vh                    NaN        1.000000 NaN     11.0 NaN   
            dist-hls-gen                NaN        1.000000 NaN    108.0 NaN   
            dist-hls-veg                NaN        1.000000 NaN    108.0 NaN   
            log_ratio_vh                NaN        0.363636 NaN     11.0 NaN   

                           supp_prod_change     supp_prod_change_perc      \
                                       mean std                  mean std   
change_type metric                                                          
Clear cut   cusum_prob_max              7.0 NaN              0.304348 NaN   
            cusum_vh                    0.0 NaN              0.000000 NaN   
            dist-hls-gen                6.0 NaN              0.029412 NaN   
            dist-hls-veg                0.0 NaN              0.000000 NaN   
            log_ratio_vh                7.0 NaN              0.304348 NaN   

                           count  
                                  
change_type metric                
Clear cut   cusum_prob_max     1  
            cusum_vh           1  
            dist-hls-gen       1  
            dist-hls-veg       1  
            log_ratio_vh       1  

[5 rows x 23 columns]

In [72]:
df_reindexed = df_dist_agg_by_type.reindex(pd.MultiIndex.from_product([df_dist_agg_by_type.index.levels[0], METRIC_ORDER], names=df_dist_agg_by_type.index.names))
df_reindexed

jaccard_score     total_samples      \
                                        mean std          mean std   
change_type metric                                                   
Clear cut   dist-hls-veg            0.000000 NaN         204.0 NaN   
            dist-hls-gen            0.000000 NaN         204.0 NaN   
            cusum_prob_max          0.636364 NaN          23.0 NaN   
            cusum_vh                0.000000 NaN          23.0 NaN   
            log_ratio_vh            0.636364 NaN          23.0 NaN   
...                                      ...  ..           ...  ..   
Wetland     log_ratio_vh            0.000000 NaN          32.0 NaN   
            mahalanobis_1d_max      0.000000 NaN          32.0 NaN   
            mahalanobis_2d          1.000000 NaN          32.0 NaN   
            mahalanobis_vh          0.000000 NaN          32.0 NaN   
            transformer             1.000000 NaN          32.0 NaN   

                               total_accuracy     precision        recall      \
                                         mean std      mean std      mean std   
change_type metric                                                              
Clear cut   dist-hls-veg             0.470588 NaN       1.0 NaN  0.000000 NaN   
            dist-hls-gen             0.441176 NaN       0.0 NaN  0.000000 NaN   
            cusum_prob_max           0.826087 NaN       1.0 NaN  0.636364 NaN   
            cusum_vh                 0.521739 NaN       1.0 NaN  0.000000 NaN   
            log_ratio_vh             0.826087 NaN       1.0 NaN  0.636364 NaN   
...                                       ...  ..       ...  ..       ...  ..   
Wetland     log_ratio_vh             0.843750 NaN       0.0 NaN  1.000000 NaN   
            mahalanobis_1d_max       0.000000 NaN       0.0 NaN  1.000000 NaN   
            mahalanobis_2d           1.000000 NaN       1.0 NaN  1.000000 NaN   
            mahalanobis_vh           0.000000 NaN       0.0 NaN  1.000000 NaN   
            transformer              1.000000 NaN       1.0 NaN  1.000000 NaN   

                                ... commission_error ommission_error      \
                                ...              std            mean std   
change_type metric              ...                                        
Clear cut   dist-hls-veg        ...              NaN        1.000000 NaN   
            dist-hls-gen        ...              NaN        1.000000 NaN   
            cusum_prob_max      ...              NaN        0.363636 NaN   
            cusum_vh            ...              NaN        1.000000 NaN   
            log_ratio_vh        ...              NaN        0.363636 NaN   
...                             ...              ...             ...  ..   
Wetland     log_ratio_vh        ...              NaN        0.000000 NaN   
            mahalanobis_1d_max  ...              NaN        0.000000 NaN   
            mahalanobis_2d      ...              NaN        0.000000 NaN   
            mahalanobis_vh      ...              NaN        0.000000 NaN   
            transformer         ...              NaN        0.000000 NaN   

                               supp_val     supp_prod_change      \
                                   mean std             mean std   
change_type metric                                                 
Clear cut   dist-hls-veg          108.0 NaN              0.0 NaN   
            dist-hls-gen          108.0 NaN              6.0 NaN   
            cusum_prob_max         11.0 NaN              7.0 NaN   
            cusum_vh               11.0 NaN              0.0 NaN   
            log_ratio_vh           11.0 NaN              7.0 NaN   
...                                 ...  ..              ...  ..   
Wetland     log_ratio_vh            0.0 NaN              5.0 NaN   
            mahalanobis_1d_max      0.0 NaN             32.0 NaN   
            mahalanobis_2d          0.0 NaN              0.0 NaN   
            mahalanobis_vh        

In [73]:
cm = sns.light_palette("blue", as_cmap=True)
df_dist_agg_by_type_styled = df_reindexed.style.background_gradient(cmap=cm)
df_dist_agg_by_type_styled

In [57]:
 df_dist_agg_by_type_styled.to_excel(across_sites_dir / 'dist_all_sites_by_type.xlsx')

# By Change Label

In [58]:
df_dist_agg_by_label = df_dist_m[['change_label', 'metric'] + AGG_COLUMNS].groupby(['change_label', 'metric']).agg(['mean', 'std'])
df_dist_agg_by_label.head()

jaccard_score           total_samples             \
                                     mean       std          mean        std   
change_label metric                                                            
OCmaj        cusum_prob_max      0.166667  0.408248     21.833333  13.482087   
             cusum_vh            0.333333  0.516398     21.833333  13.482087   
             dist-hls-gen        0.333333  0.577350    167.666667  53.519467   
             dist-hls-veg        0.333333  0.577350    167.666667  53.519467   
             log_ratio_vh        0.000000  0.000000     21.833333  13.482087   

                            total_accuracy           precision            \
                                      mean       std      mean       std   
change_label metric                                                        
OCmaj        cusum_prob_max       0.632639  0.343354  0.333333  0.516398   
             cusum_vh             0.906944  0.073488  1.000000  0.000000   
             dist-hls-gen         0.927262  0.074375  1.000000  0.000000   
             dist-hls-veg         0.927262  0.074375  1.000000  0.000000   
             log_ratio_vh         0.526736  0.277309  0.166667  0.408248   

                               recall            ... commission_error  \
                                 mean       std  ...             mean   
change_label metric                              ...                    
OCmaj        cusum_prob_max  0.333333  0.516398  ...         0.666667   
             cusum_vh        0.333333  0.516398  ...         0.000000   
             dist-hls-gen    0.333333  0.577350  ...         0.000000   
             dist-hls-veg    0.333333  0.577350  ...         0.000000   
             log_ratio_vh    0.333333  0.516398  ...         0.833333   

                                      ommission_error             supp_val  \
                                  std            mean       std       mean   
change_label metric                                                          
OCmaj        cusum_prob_max  0.516398        0.666667  0.516398   2.166667   
             cusum_vh        0.000000        0.666667  0.516398   2.166667   
             dist-hls-gen    0.000000        0.666667  0.577350  13.666667   
             dist-hls-veg    0.000000        0.666667  0.577350  13.666667   
             log_ratio_vh    0.408248        0.666667  0.516398   2.166667   

                                       supp_prod_change            \
                                   std             mean       std   
change_label metric                                                 
OCmaj        cusum_prob_max   2.041241         3.833333  3.763863   
             cusum_vh         2.041241         0.000000  0.000000   
             dist-hls-gen    17.214335         0.000000  0.000000   
             dist-hls-veg    17.214335         0.000000  0.000000   
             log_ratio_vh     2.041241         7.000000  4.516636   

                            supp_prod_change_perc            
                                             mean       std  
change_label metric                                          
OCmaj        cusum_prob_max              0.274306  0.374633  
             cusum_vh                    0.000000  0.000000  
             dist-hls-gen                0.000000  0.000000  
             dist-hls-veg                0.000000  0.000000  
             log_ratio_vh                0.380208  0.332190  

[5 rows x 22 columns]

In [74]:
df_dist_agg_by_label_count = df_dist_m[['change_label', 'metric'] + AGG_COLUMNS].groupby(['change_label', 'metric']).size().rename(index="count")#.reset_index(drop=False)
df_dist_agg_by_label_count.head()

change_label  metric        
OCmaj         cusum_prob_max    6
              cusum_vh          6
              dist-hls-gen      3
              dist-hls-veg      3
              log_ratio_vh      6
Name: count, dtype: int64

In [75]:
df_dist_agg_by_label['count'] = df_dist_agg_by_label_count
df_dist_agg_by_label

jaccard_score           total_samples  \
                                         mean       std          mean   
change_label metric                                                     
OCmaj        cusum_prob_max          0.166667  0.408248     21.833333   
             cusum_vh                0.333333  0.516398     21.833333   
             dist-hls-gen            0.333333  0.577350    167.666667   
             dist-hls-veg            0.333333  0.577350    167.666667   
             log_ratio_vh            0.000000  0.000000     21.833333   
             mahalanobis_1d_max      0.093056  0.073488     21.833333   
             mahalanobis_2d          0.166667  0.408248     21.833333   
             mahalanobis_vh          0.093056  0.073488     21.833333   
             transformer             0.333333  0.516398     21.833333   
OCmin        cusum_prob_max          1.000000  0.000000     28.400000   
             cusum_vh                1.000000  0.000000     28.400000   
             dist-hls-gen            1.000000  0.000000    182.666667   
             dist-hls-veg            1.000000  0.000000    182.666667   
             log_ratio_vh            0.600000  0.507093     28.400000   
             mahalanobis_1d_max      0.000000  0.000000     28.400000   
             mahalanobis_2d          0.800000  0.414039     28.400000   
             mahalanobis_vh          0.000000  0.000000     28.400000   
             transformer             0.933333  0.258199     28.400000   
VLmaj        cusum_prob_max          0.360606  0.451547     27.840000   
             cusum_vh                0.280000  0.458258     27.840000   
             dist-hls-gen            0.071429  0.267261    189.857143   
             dist-hls-veg            0.071429  0.267261    189.857143   
             log_ratio_vh            0.235052  0.324970     27.840000   
             mahalanobis_1d_max      0.289574  0.307936     27.840000   
             mahalanobis_2d          0.352232  0.396279     27.840000   
             mahalanobis_vh          0.289574  0.307936     27.840000   
             transformer             0.379859  0.440858     27.840000   
VLmin        cusum_prob_max          0.863636  0.351250     27.272727   
             cusum_vh                0.909091  0.294245     27.227273   
             dist-hls-gen            0.909091  0.301511    168.000000   
             dist-hls-veg            0.909091  0.301511    168.000000   
             log_ratio_vh            0.681818  0.476731     27.272727   
             mahalanobis_1d_max      0.029720  0.096378     27.227273   
             mahalanobis_2d          0.772727  0.428932     27.272727   
             mahalanobis_vh          0.029720  0.096378     27.227273   
             transformer             0.909091  0.294245     27.272727   
noChange     cusum_prob_max          0.868421  0.342570     31.500000   
             cusum_vh                1.000000  0.000000     31.473684   
             dist-hls-gen            0.954545  0.213201    192.818182   
             dist-hls-veg            1.000000  0.000000    192.818182   
             log_ratio_vh            0.868421  0.342570     31.473684   
             mahalanobis_1d_max      0.000000  0.000000     31.473684   
             mahalanobis_2d          0.921053  0.273276     31.500000   
             mahalanobis_vh          0.000000  0.000000     31.473684   
             transformer             1.000000  0.000000     31.184211   

                                           total_accuracy           precision  \
                                       std           mean       std      mean   
change_label metric                                                             
OCmaj        cusum_prob_max      13.482087       0.632639  0.343354  0.333333   
             cusum_vh            13.482087       0.906944  0.073488  1.000000   
             dist-hls-gen        53.519467       0.927262  0.074375  1.000000   
             dist-hls-veg        53.51946

In [76]:
df_reindexed = df_dist_agg_by_label.reindex(pd.MultiIndex.from_product([df_dist_agg_by_label.index.levels[0], METRIC_ORDER], names=df_dist_agg_by_label.index.names))
df_reindexed

jaccard_score           total_samples  \
                                         mean       std          mean   
change_label metric                                                     
OCmaj        dist-hls-veg            0.333333  0.577350    167.666667   
             dist-hls-gen            0.333333  0.577350    167.666667   
             cusum_prob_max          0.166667  0.408248     21.833333   
             cusum_vh                0.333333  0.516398     21.833333   
             log_ratio_vh            0.000000  0.000000     21.833333   
             mahalanobis_1d_max      0.093056  0.073488     21.833333   
             mahalanobis_2d          0.166667  0.408248     21.833333   
             mahalanobis_vh          0.093056  0.073488     21.833333   
             transformer             0.333333  0.516398     21.833333   
OCmin        dist-hls-veg            1.000000  0.000000    182.666667   
             dist-hls-gen            1.000000  0.000000    182.666667   
             cusum_prob_max          1.000000  0.000000     28.400000   
             cusum_vh                1.000000  0.000000     28.400000   
             log_ratio_vh            0.600000  0.507093     28.400000   
             mahalanobis_1d_max      0.000000  0.000000     28.400000   
             mahalanobis_2d          0.800000  0.414039     28.400000   
             mahalanobis_vh          0.000000  0.000000     28.400000   
             transformer             0.933333  0.258199     28.400000   
VLmaj        dist-hls-veg            0.071429  0.267261    189.857143   
             dist-hls-gen            0.071429  0.267261    189.857143   
             cusum_prob_max          0.360606  0.451547     27.840000   
             cusum_vh                0.280000  0.458258     27.840000   
             log_ratio_vh            0.235052  0.324970     27.840000   
             mahalanobis_1d_max      0.289574  0.307936     27.840000   
             mahalanobis_2d          0.352232  0.396279     27.840000   
             mahalanobis_vh          0.289574  0.307936     27.840000   
             transformer             0.379859  0.440858     27.840000   
VLmin        dist-hls-veg            0.909091  0.301511    168.000000   
             dist-hls-gen            0.909091  0.301511    168.000000   
             cusum_prob_max          0.863636  0.351250     27.272727   
             cusum_vh                0.909091  0.294245     27.227273   
             log_ratio_vh            0.681818  0.476731     27.272727   
             mahalanobis_1d_max      0.029720  0.096378     27.227273   
             mahalanobis_2d          0.772727  0.428932     27.272727   
             mahalanobis_vh          0.029720  0.096378     27.227273   
             transformer             0.909091  0.294245     27.272727   
noChange     dist-hls-veg            1.000000  0.000000    192.818182   
             dist-hls-gen            0.954545  0.213201    192.818182   
             cusum_prob_max          0.868421  0.342570     31.500000   
             cusum_vh                1.000000  0.000000     31.473684   
             log_ratio_vh            0.868421  0.342570     31.473684   
             mahalanobis_1d_max      0.000000  0.000000     31.473684   
             mahalanobis_2d          0.921053  0.273276     31.500000   
             mahalanobis_vh          0.000000  0.000000     31.473684   
             transformer             1.000000  0.000000     31.184211   

                                           total_accuracy           precision  \
                                       std           mean       std      mean   
change_label metric                                                             
OCmaj        dist-hls-veg        53.519467       0.927262  0.074375  1.000000   
             dist-hls-gen        53.519467       0.927262  0.074375  1.000000   
             cusum_prob_max      13.482087       0.632639  0.343354  0.333333   
             cusum_vh            13.48208

In [77]:
cm = sns.light_palette("blue", as_cmap=True)
df_dist_agg_by_label_styled = df_reindexed.style.background_gradient(cmap=cm)
df_dist_agg_by_type_styled

In [79]:
 df_dist_agg_by_type_styled.to_excel(across_sites_dir / 'dist_all_sites_by_label.xlsx')